<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_test_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a test-report

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

In this tutorial you'll learn how you can easily generate a test-report from raw data. You'll learn:

- How to create a `Document`
- How to add `Page` objects to a `Document`
- How to add `Paragraph` objects to a `Page`
- How to easily generate a `Table`  

and more

Let's start by installing `borb`

In [125]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Next we're going to create some dummy data. In real life, you would of course get this data from running your test-suite.

In [126]:
import typing

# This variable is going to represent our test data.
# It maps the name of a test to all its previous executions.
# Each execution is represented by 
#   - date of previous execution 
#   - how long it took the test to execute
#   - whether the test passed
# The timestamps here represent 8:00 AM, 8th of August, 2022 --> 8:00 AM, 12th of August, 2022
test_data: typing.Dict[str, typing.List[typing.Tuple[float, float, bool]]] = {
    "test 1": [(1659938400, 10, True), (1660024800, 9, True), (1660111200, 12, True), (1660197600, 7, True), (1660284000, 9, False)],
    "test 2": [(1659938400, 15, True), (1660024800, 16, False), (1660111200, 12, False), (1660197600, 15, True), (1660284000, 15, True)],
    "test 3": [(1659938400, 9, False), (1660024800, 8, True), (1660111200, 8, True), (1660197600, 11, True), (1660284000, 10, True)]
}


With that out of the way, we can now start generating a PDF.
We need to do some small stuff to set up everything. These steps are always the same when you're creating a basic PDF using `borb`.

In [127]:
from borb.pdf import Document
from borb.pdf import Page

# create Document
doc: Document = Document()

# create empty Page
page: Page = Page()
doc.add_page(page)

{<borb.io.read.types.Name at 0x7f7a4f868a50>: {<borb.io.read.types.Name at 0x7f7a50b01590>: Decimal('0'),
  <borb.io.read.types.Name at 0x7f7a50b017d0>: {<borb.io.read.types.Name at 0x7f7a50b01b10>: {<borb.io.read.types.Name at 0x7f7a532c6f50>: {<borb.io.read.types.Name at 0x7f7a4f82dd10>: Decimal('1'),
     <borb.io.read.types.Name at 0x7f7a4f82d810>: [{<borb.io.read.types.Name at 0x7f7a4f868b10>: [Decimal('0'),
        Decimal('0'),
        Decimal('595'),
        Decimal('842')],
       <borb.io.read.types.Name at 0x7f7a54137dd0>: {...},
       <borb.io.read.types.Name at 0x7f7a4f8688d0>: <borb.io.read.types.Name at 0x7f7a4f868190>}],
     <borb.io.read.types.Name at 0x7f7a532c6e90>: <borb.io.read.types.Name at 0x7f7a50b70450>},
    <borb.io.read.types.Name at 0x7f7a50b01910>: <borb.io.read.types.Name at 0x7f7a50b01990>}}}}

In order to easily add content to a `Page` we are going to define a `PageLayout`. A `PageLayout` is responsible for keeping track of where previous content was added, and where any new content would be placed.

In [128]:
from borb.pdf import PageLayout
from borb.pdf import SingleColumnLayout

# create PageLayout
layout: PageLayout = SingleColumnLayout(page)

Now let's add a title and subtitle for our test report.

In [129]:
from borb.pdf import Paragraph
from borb.pdf import HexColor
from decimal import Decimal

# title
layout.add(Paragraph("Test Report", 
                     font="Helvetica-Bold",
                     font_size=Decimal(20), 
                     font_color=HexColor("#489FB5")))

# subtitle
layout.add(Paragraph("12-08-2022", 
                     font_size=Decimal(16), 
                     font_color=HexColor("#16697A")))

In [130]:
from borb.pdf import TableUtil

# subtitle
layout.add(Paragraph("Execution Time (s)", 
                     font_size=Decimal(14), 
                     font_color=HexColor("#82C0CC")))

# use TableUtil to easily create a Table
layout.add(TableUtil.from_2d_array([["Test Name", "08-08-2022", "09-08-2022", "10-08-2022", "11-08-2022", "12-08-2022"],
                                    ["test 1", test_data["test 1"][0][1], test_data["test 1"][1][1], test_data["test 1"][2][1] ,test_data["test 1"][3][1], test_data["test 1"][4][1]],
                                    ["test 2", test_data["test 2"][0][1], test_data["test 2"][1][1], test_data["test 2"][2][1] ,test_data["test 2"][3][1], test_data["test 2"][4][1]],
                                    ["test 3", test_data["test 3"][0][1], test_data["test 3"][1][1], test_data["test 3"][2][1] ,test_data["test 3"][3][1], test_data["test 3"][4][1]],
                                    ], 
                                   flexible_column_width=False).set_borders_on_all_cells(True, False, True, False))

That should give the reader a quick overview. But we can do better.
Let's add a few aggregate statistics for each test.

In [131]:
from borb.pdf import FixedColumnWidthTable
from borb.pdf import Table
from borb.pdf import TableCell
from borb.pdf.canvas.layout.shape.progressbar import ProgressSquare

import numpy as np

for k,v in test_data.items():

  # subtitle
  layout.add(Paragraph(k, 
                      font_size=Decimal(14), 
                      font_color=HexColor("#82C0CC")))
  
  # avg execution time
  avg_execution_time: float = round(np.average([x[1] for x in v]), 2)
  dev_execution_time: float = round(np.std([x[1] for x in v]), 2)
  count_pass: int = len([1 for x in v if x[2]])
  count_fail: int = len(v) - count_pass

  # info
  table: Table = FixedColumnWidthTable(number_of_columns=3, number_of_rows=4)
  table.add(Paragraph("Avg. Execution Time"))
  table.add(TableCell(Paragraph(str(avg_execution_time)), col_span=2))

  # std dev
  table.add(Paragraph("Std. Dev. Execution Time"))
  table.add(TableCell(Paragraph(str(dev_execution_time)), col_span=2))

  # outliers
  count_outliers: int = len([x for x in v if (x[1] > avg_execution_time + dev_execution_time) or (x[1] < avg_execution_time - dev_execution_time)])
  table.add(Paragraph("Time Outliers"))
  table.add(Paragraph("%d / %d" % (count_outliers, len(v))))
  table.add(ProgressSquare(count_outliers / len(v), stroke_color=HexColor("#82C0CC"), fill_color=HexColor("#EDE7E3")))

  # success rate
  table.add(Paragraph("Success Rate"))
  table.add(Paragraph("%d / %d" % (count_pass, len(v))))
  table.add(ProgressSquare(count_pass / len(v), stroke_color=HexColor("#82C0CC"), fill_color=HexColor("#EDE7E3")))

  # global table properties
  table.set_padding_on_all_cells(Decimal(2), Decimal(2), Decimal(2), Decimal(2))
  table.set_borders_on_all_cells(True, False, True, False)

  layout.add(table)

Finally, we can store the `Document`.

In [132]:
from borb.pdf import PDF

# use PDF.dumps
with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, doc)